In [44]:
import collections
Individual = collections.namedtuple('Individual', ['Name', 'Marks', 'WebAssign', 'RawData'])
Marks = collections.namedtuple('Marks', ['Score', 'NumberOfNs'])

## Key      |   Data
## ---------+---------
## Name     |  full name:  last, first
## Marks    |  a namedtuple of type Marks
## WebAssign|  WebAssign score (whole number, max 100)
## Raw Data |  full pandas Series whose index is the Name above


The threshold variables are html tables which specify the required thresholds for getting a particular letter grade.

If the grading thresholds change (in a future semester, perhaps), adjust these tables in addition to the code.

In [45]:
final_threshold = "<table class='paleGrayRows'><thead> <tr> <th>Final Letter Grade</th> <th>Points Needed</th> <th>WebAssign Homework (entire course)</th> </tr> </thead> <tbody> <tr> <td>A</td> <td>≥ 46 points and no scores below G</td> <td>≥ 80%</td> </tr> <tr> <td>B</td> <td>≥ 39 points and at most 2 scores below G</td> <td>≥ 70%</td> </tr> <tr> <td>C</td> <td>≥ 35 points and at most 6 scores below G</td> <td>≥ 60%</td> </tr> <tr> <td>D</td> <td>≥ 30 points</td> <td>≥ 50%</td> </tr> <tr> <td>F</td> <td>&lt; 30 points</td> <td>&lt; 50%</td> </tr> </tbody> </table>"
midterm_threshold = "<table class='paleGrayRows'> <thead> <tr> <th>Midterm Letter Grade</th> <th>Course Standards</th> <th>WebAssign Homework (12.1-12.4 Chapter 6, Chapter 7, 8.1, 8.2)</th> </tr> </thead> <tbody> <tr> <td>A</td> <td>≥ 19 points and no scores below G</td> <td>≥ 80%</td> </tr> <tr> <td>B</td> <td>≥ 17 points and at most 2 scores below G</td> <td>≥ 70%</td> </tr> <tr> <td>C</td> <td>≥ 13 points and at most 4 scores below G</td> <td>≥ 60%</td> </tr> <tr> <td>D</td> <td>≥ 11 points</td> <td>≥ 50%</td> </tr> <tr> <td>F</td> <td>&lt; 11 points</td> <td>&lt; 50%</td> </tr> </tbody> </table>"

midterm = { "A": {"Points": 19, "Below G": 0, "WebAssign": 80},
           "B": {"Points": 17, "Below G": 2, "WebAssign": 70},
           "C": {"Points": 13, "Below G": 4, "WebAssign": 60},
           "D": {"Points": 11, "Below G": 1000, "WebAssign": 50},  
           "F": {"Points": 0, "Below G": 1000, "WebAssign": 0}
           # yes, I'm using 1000 as a threshold. I'm a teacher, not a programmer...
          }

final = { "A": {"Points": 46, "Below G": 0, "WebAssign": 80},
         "B": {"Points": 39, "Below G": 2, "WebAssign": 70},
         "C": {"Points": 35, "Below G": 6, "WebAssign": 60},
         "D": {"Points": 30, "Below G": 1000, "WebAssign": 50},
         "F": {"Points": 0, "Below G": 1000, "WebAssign": 0}
        }

## This is really dumb. I should do this differently. Want to suggest a better way? (After you've studied for your calculus test!)
def decide_letter_grade(midterm_or_final, individual):
    if midterm_or_final == "Midterm":
        if individual.Marks.Score >= midterm['A']["Points"] and individual.Marks.NumberOfNs <= midterm["A"]["Below G"] and individual.WebAssign >= midterm["A"]["WebAssign"]:
            return "A"
        elif individual.Marks.Score >= midterm['B']["Points"] and individual.Marks.NumberOfNs <= midterm["B"]["Below G"] and individual.WebAssign >= midterm["B"]["WebAssign"]:
            return "B"
        elif individual.Marks.Score >= midterm['C']["Points"] and individual.Marks.NumberOfNs <= midterm["C"]["Below G"] and individual.WebAssign >= midterm["C"]["WebAssign"]:
            return "C"
        elif individual.Marks.Score >= midterm['D']["Points"] and individual.Marks.NumberOfNs <= midterm["D"]["Below G"] and individual.WebAssign >= midterm["D"]["WebAssign"]:
            return "D"
        elif individual.Marks.Score >= midterm['F']["Points"] and individual.Marks.NumberOfNs <= midterm["F"]["Below G"] and individual.WebAssign >= midterm["F"]["WebAssign"]:
            return "F"
        else:
            return "None of the above?!?"
    elif midterm_or_final == "Final":
        if individual.Marks.Score >= final['A']["Points"] and individual.Marks.NumberOfNs <= final["A"]["Below G"] and individual.WebAssign >= final["A"]["WebAssign"]:
            return "A"
        elif individual.Marks.Score >= final['B']["Points"] and individual.Marks.NumberOfNs <= final["B"]["Below G"] and individual.WebAssign >= final["B"]["WebAssign"]:
            return "B"
        elif individual.Marks.Score >= final['C']["Points"] and individual.Marks.NumberOfNs <= final["C"]["Below G"] and individual.WebAssign >= final["C"]["WebAssign"]:
            return "C"
        elif individual.Marks.Score >= final['D']["Points"] and individual.Marks.NumberOfNs <= final["D"]["Below G"] and individual.WebAssign >= final["D"]["WebAssign"]:
            return "D"
        elif individual.Marks.Score >= final['F']["Points"] and individual.Marks.NumberOfNs <= final["F"]["Below G"] and individual.WebAssign >= final["F"]["WebAssign"]:
            return "F"
        else:
            return "None of the above?!?"
    else:
        raise ValueError("What are you doing? Midterm or Final. That's it, man")


Here, we feed our data into an HTML template and use *weasyprint* to convert the HTML into a pretty PDF.

In [46]:
from jinja2 import Environment, FileSystemLoader
from weasyprint import HTML
from datetime import date

def print_the_report_for(individual):
    today = date.today().strftime("%B %d, %Y")
    env = Environment(loader=FileSystemLoader('.'))
    template = env.get_template("template.html")
    template_vars = {
        "Name": individual.Name,
        "raw_data": individual.RawData.to_html(index=False, bold_rows=False, classes=["paleGrayRows"]),
        "score": individual.Marks.Score,
        "number_of_ns": individual.Marks.NumberOfNs,
        "webassign_grade": individual.WebAssign,
        "letter_grade": decide_letter_grade(midtermOrFinal, individual),
        "date": today,
        "midterm_or_final": midtermOrFinal,
        "grade_threshold": midterm_threshold if midtermOrFinal=="Midterm" else final_threshold,
        "extra_text": extra_text if extra_text else ""
    }

    output = template.render(template_vars)
    HTML(string=output).write_pdf("./reports/"+individual.Name+".pdf",stylesheets=["./report.css"])
   

Read in the data from the Blackboard exported CSV. 


**Future note** if blackboard columns change, this will be rather broken.

If you're just playing with this, you can read *test.csv* for test data.

In [47]:
import pandas as pd

filename_change_if_youre_testing = "lastweek.csv"

df = pd.read_csv(filename_change_if_youre_testing)
better_cols = ['Last Name', 'First Name', 'Username', 'Student ID', 'Last Access', 'Availability', 
               '01', 'V1', 'V2', 'V3', 'V4', 
               'F1', 'F2', 'F3', 'F4', 
               'I1', 'I2', 'I3', 'I4', 'I5', 'I6', 
               'A1', 'A2', 
               'C1', 'C2', 'C3', 'C4', 'C5',
               'S1', 'S2', 'S3/S4', '--', 'S5', 'S6', 'S7', 
               'P1', 'P2', 'P3', 'WebAssign']
df.columns = better_cols

## Make a better index column and dump everything not grade-related
df['Name'] = df['Last Name'] + ', ' + df['First Name']
df.drop(['Last Name', 'First Name', 'Username', 'Student ID', 'Last Access', 'Availability'], axis=1, inplace=True)
df.set_index("Name", inplace=True)


Define the tools for reading the S, G, and N strings into their corresponding numeric values

In [48]:
def sgn_to_number(sgn):
#Note that S has the value 1, G has the value 0.5, and N has the value 0; else is 0; 
    if (sgn == "S"):
        return 1
    elif (sgn == "G"):
        return 0.5
    else:
        return 0
    
def marks_to_score(marks):
    ## First, extract the last two entries:
    try: 
        marks = marks[-2:]
    except TypeError:  # encountered NaN or not a string
        return 0
    except IndexError: # encountered NaN; bah
        return 0
    
    # this needs to be a dictionary, not a Marks namedtuple
    return {'score' : sum(map(sgn_to_number, marks)), 
                'number of ns' : marks.count("N")+marks.count("Z")}

def summative(marks):
    score_so_far = 0
    ns_so_far = 0
    for cell in marks:
        try: 
            score_so_far += cell['score']
            ns_so_far += cell['number of ns']
        except:
            pass
    return Marks(Score=score_so_far, NumberOfNs=ns_so_far)

Iterate over the dataset, sum marks, assign webassign grade. 

Note: Iterating over a *pandas* DataFrame is a bad idea for large datasets. If you're a student (not me), learn to do this differently. For this use case of no more that 40 records, I'm fine with performance loss.

In [49]:
class_scores = list()
for index, row in df.iterrows():
    raw = pd.DataFrame({'Standard':row.index, 'Marks':row.values})
    raw = raw.dropna()
    class_scores.append( Individual(Name=index, 
                                    Marks=summative(row.apply(marks_to_score)), 
                                    WebAssign=row.loc['WebAssign'], 
                                    RawData=raw) )


In [50]:
# temporary stuff:
midtermOrFinal = "Final"
extra_text = "<h2>Last week before final</h2><p> <b>Note:</b> this is prepared the week before the final. There is still one more quiz on one standard; you can get up to 2 points (two S's) or as low as 2 additional N's. Note also the grade thresholds that must be met on the next page.</p> <p> Unhappy with your grade? There's still time to reassess - break week, last week of classes, and during finals week. Additionally, the final exam will cover the second half material. </p> <p>Happy with your grade? Do well on the last quiz and you can skip the final.</p>"

for individual in class_scores: 
    print_the_report_for(individual)